In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

In [3]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp:int) -> str:
        return 'ComputeClass: ' + tmp + 'd'

In [4]:
def input_handler(i):
    return [i] * 2

In [5]:
out_queue= ppq.ThreadQueue().register_callback(print)
temp = partials(print, 'test')

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), input_handler, distributor),
            (input_handler, compute, ppq.ThreadQueue()),
                (compute, print, ppq.ThreadQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.ThreadQueue()),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.ThreadQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

[ThreadQueue(name=None, runner=1, id=135350332057360)]

  |->input_handler(i:Any)->Any(id=135350332297024)-IteratorDistributor

    |->[ThreadQueue(name=None, runner=1, id=135350326241680)]

      |->compute(x:Any)->Any(id=135350332303744)

        |->[ThreadQueue(name=None, runner=1, id=135350332328096)]

          |->print(*args:Any, sep:Any=str, end:Any=str, file:Any=NoneType, flush:Any=bool)->Any(id=135350541771824)

        |->[ThreadQueue(name=None, runner=1, id=135350459797856)]

        |->[ThreadQueue(name=None, runner=1, id=135350332327488)]

          |->compute2(x:Any)->Any(id=135350332607200)

            |->[ThreadQueue(name=None, runner=1, id=135350332328096)]

              |->print(*args:Any, sep:Any=str, end:Any=str, file:Any=NoneType, flush:Any=bool)->Any(id=135350541771824)...

            |->[ThreadQueue(name=None, runner=1, id=135350459797856)]

            |->[ThreadQueue(name=None, runner=1, id=135350326130256)]

              |->ComputeClass[(*args:Any, **kwds:

In [6]:
with flow:
   for i in range(10):
      flow.put(i)

compute: 0
compute: 0
compute2: compute: 0c
ComputeClass: compute2: compute: 0cd
compute: 1
compute2: compute: 0c
ComputeClass: compute2: compute: 0cd
compute: 1
compute2: compute: 1c
ComputeClass: compute2: compute: 1cd
compute: 2
compute2: compute: 1c
ComputeClass: compute2: compute: 1cd
compute: 2
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute: 3
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute: 3
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 4
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 4
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 5
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 5
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 6
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 6
compute2: compute: 6c
ComputeClass: compute2: compute: 6cd
compute: 7
compute2: compute: 6c
ComputeClass: compute2: compute: 6cd
compute: 

In [7]:
flow.alive()

False

In [8]:
out_queue.run()

compute: 0
compute: 0
compute2: compute: 0c
ComputeClass: compute2: compute: 0cd
compute: 1
compute2: compute: 0c
ComputeClass: compute2: compute: 0cd
compute: 1
compute2: compute: 1c
ComputeClass: compute2: compute: 1cd
compute: 2
compute2: compute: 1c
ComputeClass: compute2: compute: 1cd
compute: 2
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute: 3
compute2: compute: 2c
ComputeClass: compute2: compute: 2cd
compute: 3
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 4
compute2: compute: 3c
ComputeClass: compute2: compute: 3cd
compute: 4
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 5
compute2: compute: 4c
ComputeClass: compute2: compute: 4cd
compute: 5
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 6
compute2: compute: 5c
ComputeClass: compute2: compute: 5cd
compute: 6
compute2: compute: 6c
ComputeClass: compute2: compute: 6cd
compute: 7
compute2: compute: 6c
ComputeClass: compute2: compute: 6cd
compute: 

ThreadQueue[()->Any](
  name=None,
  num_runner=1,
  timeout=None,
)

In [9]:
out_queue.release()